In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, Subset
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
import random

In [2]:
# --- 1. Setup ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. Load Dataset (e.g., CIFAR-10)

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
sample_indices = random.sample(range(len(test_dataset)), 1000)
test_subset = Subset(test_dataset, sample_indices)
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)

# 3. AlexNet

## Set Up Pretrained AlexNet

In [ ]:
alexnet = models.alexnet(pretrained=True)
alexnet.eval()
alexnet.to(device)

## Evaluate AlexNet + Extract Features

In [ ]:
alexnet_correct = 0
alexnet_total = 0
features_list = []
labels_list = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = alexnet(images)
        _, preds = torch.max(outputs, 1)
        alexnet_total += labels.size(0)
        alexnet_correct += (preds == labels).sum().item()

        # Extract penultimate layer features
        penultimate = alexnet.classifier[:-1]
        feats = penultimate(alexnet.features(images).view(images.size(0), -1))
        features_list.append(feats.cpu().numpy())
        labels_list.append(labels.cpu().numpy())

alexnet_accuracy = alexnet_correct / alexnet_total
print(f"AlexNet Accuracy: {alexnet_accuracy:.4f}")

# 4. SVM

## Prepare Features for SVM



In [9]:
X = np.vstack(features_list)
y = np.hstack(labels_list)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## Train + Evaluate SVM

In [ ]:
svm = SVC(kernel='linear')
svm.fit(X_scaled, y)
y_pred = svm.predict(X_scaled)
svm_accuracy = accuracy_score(y, y_pred)
print(f"SVM Accuracy (on same features): {svm_accuracy:.4f}")

# 5. Comparison

In [ ]:
if alexnet_accuracy > svm_accuracy:
    print("✅ AlexNet outperforms the SVM.")
else:
    print("⚠️ SVM performs comparably or better (unexpected).")